In [1]:

import sys
import pandas as pd              # python package for dataframes
import os                        # used to change directory paths
import matplotlib.pyplot as plt  # python package for plotting
import numpy as np
import seaborn as sns #package for plotting
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
from IPython.display import display, HTML  # Make tables pretty
from datetime import datetime
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn import preprocessing
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import GaussianNB
from sklearn.calibration import CalibratedClassifierCV
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
#import sklearn.cross_validation.Bootstrap as bootstrap
import scipy
import zipfile
import gzip



### import files and merge

In [2]:
path_data = "\\\cskma0294\\F\\Evaluations\\JobPath"
os.chdir(path_data)
df = pd.read_csv("jp_outcomes.csv")



# Convert floats to ints to make reporting easier
numerics = ["float64"]
for col in df.select_dtypes(include=numerics).columns:
    if df[col].isnull().sum() == 0:
        # print(col)
        df[col] = df[col].astype("int")




periods = pd.PeriodIndex(start="2016Q1", end="2017Q4", freq="Q")
period_list = list(periods.strftime("%YQ%q"))
periods_len = len(period_list)

for i, period in enumerate(reversed(period_list)):

    df[period] = df["Group" + str(periods_len - i)].str[:1]
    cat_map = {"T": 1, "C": 0, "0": -1}
    df[period] = df[period].map(cat_map)
    if i > 0:
        later_periods = period_list[-i:]
        df.loc[df[period] == 1, later_periods] = -2

df.shape

(110135, 135)

In [3]:
        
path_data = "\\\cskma0294\\F\\Evaluations\\JobPath\\Quarterly_status"
os.chdir(path_data)

df_shares = pd.read_csv("shares_2017.zip")
df=pd.merge(df, df_shares, on='ppsn', how='left')

df.head()

,Unnamed: 0,id,ppsn,cluster,age,duration_days_0,sex,LM_code_rank_P1,occupation_rank_P1,ada_code_rank_P1,...,Empl_13_15share,Empl_13_15sum,Ed_or_Training_13_15share,Ed_or_Training_13_15sum,LM_WSW_13_15share,LM_WSW_13_15sum,LR_13_15share,LR_13_15sum,WSW_13_15share,WSW_13_15sum
0,0,0,0099665M,2,54,6316,1,6,2.0,5,...,0.0,0,0.0,0,0.967,88,0.549,50,0.0,0
1,1,1,0206477C,2,59,3964,2,6,5.0,5,...,0.0,0,0.0,0,1.000,91,0.560,51,0.0,0
2,2,2,0202920T,2,59,1285,2,6,7.0,4,...,0.0,0,0.0,0,0.989,90,0.560,51,0.0,0
3,3,3,0205276K,5,55,1415,2,6,2.0,6,...,0.0,0,0.0,0,1.000,91,0.000,0,0.0,0
4,4,4,0202425J,2,59,2445,2,6,7.0,5,...,0.0,0,0.0,0,1.000,91,0.549,50,0.0,0


In [4]:
df.drop('Unnamed: 0', axis=1)



,id,ppsn,cluster,age,duration_days_0,sex,LM_code_rank_P1,occupation_rank_P1,ada_code_rank_P1,family_flag_rank_P1,...,Empl_13_15share,Empl_13_15sum,Ed_or_Training_13_15share,Ed_or_Training_13_15sum,LM_WSW_13_15share,LM_WSW_13_15sum,LR_13_15share,LR_13_15sum,WSW_13_15share,WSW_13_15sum
0,0,0099665M,2,54,6316,1,6,2.0,5,3,...,0.000,0,0.0,0,0.967,88,0.549,50,0.000,0
1,1,0206477C,2,59,3964,2,6,5.0,5,3,...,0.000,0,0.0,0,1.000,91,0.560,51,0.000,0
2,2,0202920T,2,59,1285,2,6,7.0,4,1,...,0.000,0,0.0,0,0.989,90,0.560,51,0.000,0
3,3,0205276K,5,55,1415,2,6,2.0,6,3,...,0.000,0,0.0,0,1.000,91,0.000,0,0.000,0
4,4,0202425J,2,59,2445,2,6,7.0,5,3,...,0.000,0,0.0,0,1.000,91,0.549,50,0.000,0
5,5,0509918T,2,39,1129,2,6,7.0,6,3,...,0.000,0,0.0,0,0.978,89,0.560,51,0.000,0
6,6,0511303R,2,19,466,1,6,1.0,1,1,...,0.000,0,0.0,0,0.714,65,0.165,15,0.000,0
7,7,0526622S,1,57,445,2,6,4.0,1,1,...,0.000,0,0.0,0,0.451,41,0.473,43,0.000,0
8,8,0527442V,2,59,557,1,6,10.0,6,3,...,0.121,11,0.0,0,0.769,70,0.198,18,0.000,0
9,9,0503804U,2,31,914,2,6,2.0,6,4,...,0.000,0,0.0,0,1.000,91,0.154,14,0.000,0


In [5]:
df.set_index('id', drop=True)

,Unnamed: 0,ppsn,cluster,age,duration_days_0,sex,LM_code_rank_P1,occupation_rank_P1,ada_code_rank_P1,family_flag_rank_P1,...,Empl_13_15share,Empl_13_15sum,Ed_or_Training_13_15share,Ed_or_Training_13_15sum,LM_WSW_13_15share,LM_WSW_13_15sum,LR_13_15share,LR_13_15sum,WSW_13_15share,WSW_13_15sum
id,,,,,,,,,,,,,,,,,,,,,
0,0,0099665M,2,54,6316,1,6,2.0,5,3,...,0.000,0,0.0,0,0.967,88,0.549,50,0.000,0
1,1,0206477C,2,59,3964,2,6,5.0,5,3,...,0.000,0,0.0,0,1.000,91,0.560,51,0.000,0
2,2,0202920T,2,59,1285,2,6,7.0,4,1,...,0.000,0,0.0,0,0.989,90,0.560,51,0.000,0
3,3,0205276K,5,55,1415,2,6,2.0,6,3,...,0.000,0,0.0,0,1.000,91,0.000,0,0.000,0
4,4,0202425J,2,59,2445,2,6,7.0,5,3,...,0.000,0,0.0,0,1.000,91,0.549,50,0.000,0
5,5,0509918T,2,39,1129,2,6,7.0,6,3,...,0.000,0,0.0,0,0.978,89,0.560,51,0.000,0
6,6,0511303R,2,19,466,1,6,1.0,1,1,...,0.000,0,0.0,0,0.714,65,0.165,15,0.000,0
7,7,0526622S,1,57,445,2,6,4.0,1,1,...,0.000,0,0.0,0,0.451,41,0.473,43,0.000,0
8,8,0527442V,2,59,557,1,6,10.0,6,3,...,0.121,11,0.0,0,0.769,70,0.198,18,0.000,0


In [6]:
df.drop('Unnamed: 0', axis=1,inplace = True)

### select features/variables

In [7]:
select=df[df['2016Q1']!=-1]
select['y']=df['2016Q1']
#y=['y_1']

select['y'].value_counts()
select=select.set_index('id',drop=True)
select=select.sort_index()
select=select.reset_index()

C:\Users\hughcronin\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [8]:
select.index.names=['2016Q1_index']

In [9]:
select.tail()

,id,ppsn,cluster,age,duration_days_0,sex,LM_code_rank_P1,occupation_rank_P1,ada_code_rank_P1,family_flag_rank_P1,...,Empl_13_15sum,Ed_or_Training_13_15share,Ed_or_Training_13_15sum,LM_WSW_13_15share,LM_WSW_13_15sum,LR_13_15share,LR_13_15sum,WSW_13_15share,WSW_13_15sum,y
2016Q1_index,,,,,,,,,,,,,,,,,,,,,
94343,110130,9638538J,2,58,804,1,6,10.0,1,1,...,0,0.0,0,1.000,91,0.000,0,0.0,0,0
94344,110131,9641220S,1,54,590,1,6,6.0,1,1,...,0,0.0,0,0.901,82,0.560,51,0.0,0,1
94345,110132,9641428T,2,52,2733,1,6,6.0,2,1,...,0,0.0,0,1.000,91,0.560,51,0.0,0,0
94346,110133,9641733B,3,52,396,1,6,6.0,1,1,...,0,0.0,0,0.615,56,0.000,0,0.0,0,0
94347,110134,9641815D,2,58,1458,1,6,7.0,1,1,...,0,0.0,0,1.000,91,0.352,32,0.0,0,0


In [10]:
#X = data_final.loc[:, data_final.columns != 'y']
y = select.loc[:, select.columns == 'y']

#features =['sw_pay_mean_1315', 'earn_tot_mean_1315', 'age', 'occupation_rank_P1','Duration Bands', 'cluster', 
#  'family_flag_rank_P1', 'WSW_13_15share', 'Ed_or_Training_13_15share', 'LM_WSW_13_15share']
features =['sw_pay_2013',
            'sw_pay_2014',
            'sw_pay_2015',
            'earn_tot_2012',
            'earn_tot_2013',
            'earn_tot_2014',
            'earn_tot_2015',
           'age', 
           #'occupation_rank_P1',
          'Duration Bands',
           #'LM_code_rank_P1',
          #'LM_WSW_13_15share',
           'family_flag_rank_P1',
           #'LM_WSW_13_15share',
           #'hist_lr_0',
           'total_duration_days',
           # 'Duration Bands',
           #'Empl_13_15share',
            'Empl_13_15sum',
 'Ed_or_Training_13_15share',
 #'Ed_or_Training_13_15sum',
 'LM_WSW_13_15share',
 #'LM_WSW_13_15sum',
 #'LR_13_15share',
 'LR_13_15sum',
 'WSW_13_15share',
 #'WSW_13_15sum',
           #'Ed_or_Training_13_15share',
          ]

X = select[features] # to enforce column order

In [11]:
X.shape

(94348, 16)

#### check for missing values

In [120]:
df.columns[df.isna().any()].tolist()


['occupation_rank_P1',
 'marital_status_rank_P1',
 'occupation_rank_P2',
 'marital_status_rank_P2',
 'occupation_rank_P3',
 'marital_status_rank_P3',
 'occupation_rank_P4',
 'marital_status_rank_P4',
 'occupation_rank_P5',
 'marital_status_rank_P5',
 'occupation_rank_P6',
 'marital_status_rank_P6',
 'occupation_rank_P7',
 'occupation_rank_P8',
 'marital_status_rank_P7',
 'marital_status_rank_P8',
 'Cancellationsubcategory']

#### logistic regression based on TPot range of options

In [12]:
df.sort_values(['id'])

,id,ppsn,cluster,age,duration_days_0,sex,LM_code_rank_P1,occupation_rank_P1,ada_code_rank_P1,family_flag_rank_P1,...,Empl_13_15share,Empl_13_15sum,Ed_or_Training_13_15share,Ed_or_Training_13_15sum,LM_WSW_13_15share,LM_WSW_13_15sum,LR_13_15share,LR_13_15sum,WSW_13_15share,WSW_13_15sum
0,0,0099665M,2,54,6316,1,6,2.0,5,3,...,0.000,0,0.0,0,0.967,88,0.549,50,0.000,0
1,1,0206477C,2,59,3964,2,6,5.0,5,3,...,0.000,0,0.0,0,1.000,91,0.560,51,0.000,0
2,2,0202920T,2,59,1285,2,6,7.0,4,1,...,0.000,0,0.0,0,0.989,90,0.560,51,0.000,0
3,3,0205276K,5,55,1415,2,6,2.0,6,3,...,0.000,0,0.0,0,1.000,91,0.000,0,0.000,0
4,4,0202425J,2,59,2445,2,6,7.0,5,3,...,0.000,0,0.0,0,1.000,91,0.549,50,0.000,0
5,5,0509918T,2,39,1129,2,6,7.0,6,3,...,0.000,0,0.0,0,0.978,89,0.560,51,0.000,0
6,6,0511303R,2,19,466,1,6,1.0,1,1,...,0.000,0,0.0,0,0.714,65,0.165,15,0.000,0
7,7,0526622S,1,57,445,2,6,4.0,1,1,...,0.000,0,0.0,0,0.451,41,0.473,43,0.000,0
8,8,0527442V,2,59,557,1,6,10.0,6,3,...,0.121,11,0.0,0,0.769,70,0.198,18,0.000,0
9,9,0503804U,2,31,914,2,6,2.0,6,4,...,0.000,0,0.0,0,1.000,91,0.154,14,0.000,0


In [13]:
logreg = LogisticRegression(C=25.0, dual=False, penalty="l1")
logreg.fit(X, y)

C:\Users\hughcronin\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\hughcronin\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=25.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l1', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [14]:
roc_auc_score(y, y_pred_prob)

NameError: name 'y_pred_prob' is not defined

In [ ]:



logit_roc_auc = roc_auc_score(y, logreg.predict(X))
fpr, tpr, thresholds = roc_curve(y, logreg.predict_proba(X)[:,1])
plt.figure()
plt.plot(fpr, tpr, label='Logistic Regression (area = 0.592)' )
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.savefig('Log_ROC')
plt.show()


In [15]:
a=logreg.predict_proba(X)
y_pred_prob = a[:,1]
roc_auc_score(y, y_pred_prob)
print(y_pred_prob)
type(y_pred_prob)
y_pred_prob.shape
pp= pd.Series(y_pred_prob)

[0.08740752 0.08883066 0.08955524 ... 0.08199813 0.03433574 0.07458944]


In [16]:
pp.shape

(94348,)

In [17]:
pp.tail()

94343    0.060989
94344    0.081539
94345    0.081998
94346    0.034336
94347    0.074589
dtype: float64

In [18]:
select['Propensity']=pp

In [19]:
df=select

In [20]:
df.tail()

,id,ppsn,cluster,age,duration_days_0,sex,LM_code_rank_P1,occupation_rank_P1,ada_code_rank_P1,family_flag_rank_P1,...,Ed_or_Training_13_15share,Ed_or_Training_13_15sum,LM_WSW_13_15share,LM_WSW_13_15sum,LR_13_15share,LR_13_15sum,WSW_13_15share,WSW_13_15sum,y,Propensity
2016Q1_index,,,,,,,,,,,,,,,,,,,,,
94343,110130,9638538J,2,58,804,1,6,10.0,1,1,...,0.0,0,1.000,91,0.000,0,0.0,0,0,0.060989
94344,110131,9641220S,1,54,590,1,6,6.0,1,1,...,0.0,0,0.901,82,0.560,51,0.0,0,1,0.081539
94345,110132,9641428T,2,52,2733,1,6,6.0,2,1,...,0.0,0,1.000,91,0.560,51,0.0,0,0,0.081998
94346,110133,9641733B,3,52,396,1,6,6.0,1,1,...,0.0,0,0.615,56,0.000,0,0.0,0,0,0.034336
94347,110134,9641815D,2,58,1458,1,6,7.0,1,1,...,0.0,0,1.000,91,0.352,32,0.0,0,0,0.074589


##### check of any null values in propensity

In [21]:
type(df['Propensity'] )
df['Propensity'].isnull().values.any()

#df['Propensity'].isnull().sum().sum() 

False

#### retain inital, unweighted, dataset variables to compare

In [23]:
initial=df[['Propensity','y','age', 'sw_pay_mean_1315', 'duration_days_0', 'earn_tot_mean_1315']]

In [24]:
initial.shape

(94348, 6)

In [9]:
initial.columns[initial.isna().any()].tolist()


['Propensity']

#### calculate odds ratios

In [25]:
df['o_ratio']= df['y']
df['o_ratio']= df['y'] + (1-df['y'])*df['Propensity']/(1-df['Propensity'])


In [26]:
average_odds=dict(df['o_ratio'].groupby(df['y']).mean())

df['report_o_ratio']=df['o_ratio']
df.loc[df['y']==0, 'report_o_ratio']=df['report_o_ratio']/average_odds[0]




#### age, social welfare payments, earnings and duration - compare the weighted and unweighted means

In [187]:

df['weighted_age']=df['age']*df['report_o_ratio']
print(df['age'].groupby(df['y']).mean())
print(df['weighted_age'].groupby(df['y']).mean())


df['weighted_age']=df['age']*df['report_o_ratio']
print(df['age'].groupby(df['y']).mean())
print(df['weighted_age'].groupby(df['y']).mean())

df['weighted_earn']=df['earn_tot_mean_1215']*df['report_o_ratio']
print(df['earn_tot_mean_1215'].groupby(df['y']).mean())
print(df['weighted_earn'].groupby(df['y']).mean())

df['weighted_sw_pay']=df['sw_pay_mean_1315']*df['report_o_ratio']
print(df['sw_pay_mean_1315'].groupby(df['y']).mean())
print(df['weighted_sw_pay'].groupby(df['y']).mean())

df['weighted_dur_0']=df['duration_days_0']*df['report_o_ratio']
print(df['duration_days_0'].groupby(df['y']).mean())
print(df['weighted_dur_0'].groupby(df['y']).mean())


y
0    38.885509
1    40.348145
Name: age, dtype: float64
y
0    40.335101
1    40.348145
Name: weighted_age, dtype: float64
y
0    38.885509
1    40.348145
Name: age, dtype: float64
y
0    40.335101
1    40.348145
Name: weighted_age, dtype: float64
y
0    1711.561661
1    1040.233650
Name: earn_tot_mean_1215, dtype: float64
y
0    1037.616143
1    1040.233650
Name: weighted_earn, dtype: float64
y
0    9371.743159
1    9792.790898
Name: sw_pay_mean_1315, dtype: float64
y
0    9790.090616
1    9792.790898
Name: weighted_sw_pay, dtype: float64
y
0    1557.782926
1    1757.338291
Name: duration_days_0, dtype: float64
y
0    1764.778055
1    1757.338291
Name: weighted_dur_0, dtype: float64


In [27]:
var_to_weight =['total_duration_days','sw_pay_mean_1315','earn_tot_mean_1215','duration_days_0',
       'LR_2017share','Ed_or_Training_2017share', 'earn_tot_2012', 'earn_tot_2013', 
       'earn_tot_2014', 'earn_tot_2015', 'earn_tot_2016', 'earn_tot_2017',]
#'Duration Bands',


for col in var_to_weight:
    df["w_" + col] = df[col] *df['report_o_ratio']

# for v in var_to_weight:
#     df['weighted_'+(v)]=df['v']*df['report_o_ratio']
 

In [28]:
#df.head()


print(df['w_LR_2017share'].groupby(df['y']).mean())
print(df['w_earn_tot_2017'].groupby(df['y']).mean())


y
0    0.658072
1    0.658182
Name: w_LR_2017share, dtype: float64
y
0    3250.367847
1    4506.965239
Name: w_earn_tot_2017, dtype: float64


In [32]:
group_cl=df.groupby(['y', 'cluster'])

In [36]:
group_cl['w_LR_2017share'].mean()

TypeError: 'float' object is not subscriptable

In [39]:

#non-z=df.loc[df['w_earn_tot_2017']>0]


non_Z=df[df['w_earn_tot_2017']>0]


In [42]:
#print(non_Z['w_LR_2017share'].groupby(non_Z['y', 'cluster']).mean())


In [40]:

print(non_Z['w_LR_2017share'].groupby(non_Z['y']).mean())
print(non_Z['w_earn_tot_2017'].groupby(non_Z['y']).mean())

y
0    0.395958
1    0.439305
Name: w_LR_2017share, dtype: float64
y
0     8436.661969
1    11214.165403
Name: w_earn_tot_2017, dtype: float64


In [ ]:
##### Find those who earned something in each group, T and C;

#### multiply the count in C group in employment (>0) by readjusted odds ratio, sum of those weights divided by sum of all C group weights

##### for the uplift in earnings

##### knock out the non-earners, compare average amount

In [ ]:
  

df['weighted_sw']=df[' age']*df['report_o_ratio']


print(df['age'].groupby(df['y']).mean())


print(df['weighted_age'].groupby(df['y']).mean())


'total_duration_days',
           # 'Duration Bands',
           'sw_pay_mean_1315','earn_tot_mean_1215',duration_days_0''LR_2017share',
'Ed_or_Training_2017share',

'earn_tot_2012',
 'earn_tot_2013',
 'earn_tot_2014',
 'earn_tot_2015',
 'earn_tot_2016',
 'earn_tot_2017',



In [ ]:
#df.groupby("y").std(ddof=0) 



print(df.groupby(['y'])['age'].apply(np.std))

print(df.groupby(['y'])['age'].apply(np.mean))

#### having generated weighted columns with '_weighted' appended to the var name, generate smd as the mean/std for the original variables and the weighted variables

In [ ]:
# is it possible to loop through the columns in the dataframe and calculate a value for smd
# this needs to subtract mean values where y =0 from those where y=1; then add std dev values for both y groups and divide by two
col_list=['total_duration_days','sw_pay_mean_1315','earn_tot_mean_1215','duration_days_0','LR_2017share','Ed_or_Training_2017share',
'earn_tot_2012', 'earn_tot_2013', 'earn_tot_2014', 'earn_tot_2015', 'earn_tot_2016', 'earn_tot_2017', 'LR_2017share',
          'Ed_or_Training_2017share']
for col in col_list:
    df['smd']=df.groupby(['y']).apply(np.mean)/df.groupby(['y']).apply(np.std)
            # print(col)
    df[col] = df[col].astype("float")
